In [7]:
import asyncio
import asyncpraw
import nest_asyncio
import pandas as pd
from datetime import datetime
import aiohttp

nest_asyncio.apply()

# Load CSV and filter matchdays 13 to 20
df = pd.read_csv(r"Z:\Coding_II\Reddit_Sentiment_Analysis\dataframe\full_post(finished_matches_order).csv")
filtered_df = df[(df["matchday"] >= 13) & (df["matchday"] <= 20)].dropna(subset=["submission_id"])

# Reddit API setup
async def get_reddit_instance():
    session = aiohttp.ClientSession()
    return asyncpraw.Reddit(
        client_id="j9JdlLnSugDJxrhTFoXExQ",
        client_secret="QJbefKRgxmm7fY1ZYp6oQL0iv2wgUg",
        user_agent="Hi",
        username="FRD-23",
        password="@_Zurite1721212232005",
        requestor_kwargs={"session": session}
    )

# Scrape comments from the given submission IDs
async def scrape_comments(submission_ids):
    reddit = await get_reddit_instance()
    all_comments = []
    semaphore = asyncio.Semaphore(3)  # Limit concurrent requests

    async def fetch_comments(submission_id):
        async with semaphore:
            try:
                submission = await reddit.submission(id=submission_id)
                await submission.comments.replace_more(limit=3)

                for comment in submission.comments.list():
                    all_comments.append({
                        "submission_id": submission_id,
                        "comment_id": comment.id,
                        "comment_text": comment.body,
                        "comment_author": comment.author.name if comment.author else "[deleted]",
                        "comment_score": comment.score,
                        "comment_time": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        "parent_id": comment.parent_id if comment.parent_id else "None"
                    })
            except Exception as e:
                print(f"Error fetching comments for {submission_id}: {e}")

    await asyncio.gather(*(fetch_comments(sub_id) for sub_id in submission_ids))
    return all_comments

# Run async tasks
async def main():
    submission_ids = filtered_df["submission_id"].unique()
    comments_data = await scrape_comments(submission_ids)
    
    if comments_data:
        df_comments = pd.DataFrame(comments_data)
        csv_filename = f"epl_comments_matchday13-20_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_comments.to_csv(csv_filename, index=False)
        print(f"Exported {len(comments_data)} comments to {csv_filename}")
    else:
        print("No comments extracted.")

# Run the async function
asyncio.run(main())

C:\Users\duckh\AppData\Local\Temp\ipykernel_29916\2099271362.py:45: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "comment_time": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


Error fetching comments for 1h9jagx: received 429 HTTP response
Error fetching comments for 1hjyzz8: received 429 HTTP response
Exported 27411 comments to epl_comments_matchday13-20_20250306_192231.csv
